# Quantum Resource Estimation 2021 Demo
This file contains example code and explanation of the Quantmark VQE benchmarking framework with annotation of how it works now, and what we want to add in the future.

DISCLAIMER: The package is still in very active development and might not work on every device. This demo is meant as a show of progress and is not yet ready for use.

### Add package to PYTHONPATH

The package does not yet have an installer, so let's just add it to our path for now.

In [4]:
import os, sys

sys.path.append(os.getcwd())

### Imports

In [5]:
from quantmark import Qresult
import tequila as tq
import numpy as np
import matplotlib.pyplot as plt
import itertools

# Create hydrogen molecule
Other molecules works in a similar fashion. We focus on hydrogen for now since this is a demo.

We use the Tequila library for representing the molecule, because it works together with various established libraries and it allows us to abstract away from the chemical details of VQE and focus on the algorithmic structure. 

Moreover, it gives us a unified representation of the molecules, hamiltonians, circuits, and optimizers, such that identifying the difference between various VQE algorithms is more easily identified.

In [11]:
def create_H2(R, **kwargs): 
    geometry = f'H 0.0 0.0 0.0\nH 0.0 0.0 {R}'
    return tq.chemistry.Molecule(geometry=geometry, **kwargs)

# Defining the 4 experiments
Next, we define the parameters we use for our experiments. We want to test 2 basis_sets and 2 transformations. With the chosen configurations, we will obtain circuits with different amounts of qubits for the hydrogen molecules. 

The idea is that the real-world molecule with bond distance `r` should have a particular amount of energy, regardless of our choice of basis set or transformation. However, our choices will result in 2, 4, 6, or 8 qubits, depending on the choices made. This results in a different circuit and thus a different VQE result, eventhough the molecule that is being simulated is exactly the same.

In [16]:
# Create the molecules
basis_sets = ["sto-3g", "6-31g"]
transformations = ["bravyi-kitaev", "tapered_bravyi_kitaev"]
step = 0.1
molecular_distances = np.arange(.0 + step, 1.5 + step, step)

In [21]:
molecules = {" ".join([bs, tf]):[create_H2(r, basis_set=bs, transformation=tf) for r in molecular_distances] 
             for bs, tf in itertools.product(basis_sets, transformations)}

# Calculating baseline energies
Each choice of basis set results in a slightly different energy curve for the same hydrogen molecule. This is because some basis sets describe all the interacting factors within the molecule more accurately than others. Of course, a more detailed description would result in more classical compute time and possibly more qubits on the system, hence most research is done using `STO-3G` since that basis set can be represented with the least qubits.

For fair comparisons of any VQE algorithm simulating the same molecule, we define a "gold standard" baseline. This is the energy curve that would be obtained with the most accurate classical numerical method currently available. This is not just the `fci` method for taking into account the full configuration interaction between the particles in the molecule, but also the most accurate basis set (we found `"def2-QZVPPD"`, please let me know if there is something better). Moreover, the baseline will consider the full active space of the molecule, so the core remains unfrozen and no electrons are ignored.

Ofcourse, this doesn't give enough insight into how the VQE is performing with respect its classical counter part because the VQE is probably using a completely different basis set that is much less accurate. For this purpuse, we also introduce the `experiment_truth`, this is the `fci` energy of the same molecule under the same restrictions as the VQE (i.e. the same basis_set, active space, etc.). This is the theoretical optimum of the VQE performance. If it needs to be more accurate, you would need a different basis_set or more qubits to represent all the electrons.

Lastly, we want to introduce an upperbound for the VQE performance where we can say for certain that the algorithm is performing poorly. For this, we use the `hf` method. The Hartree-fock method for calculating the molecular energy is a rather crude approximation of the energy that only takes into account a single slater determinant. As such, it doesn't take into account the interactions between the electrons in the molecule, which makes the calculations much simpler numerically, but the resulting energy is not very accurately. A good VQE algorithm should at least be able to take into account the interactions between electrons. We call this baseline the `experiment_approximation` and it is the same as the experiment_truth except that is using Hartree-Fock instead of the full configuration interaction as numerical method.

In [15]:
def gold_standard(distances):
    return baseline(distances, "def2-QZVPPD")
    
def baseline(distances, basis_set, numerical_method="fci"):
    return [create_H2(r, basis_set="def2-QZVPPD").compute_energy(numerical_method) for r in distances]

In [19]:
ground_truth = gold_standard(molecular_distances)
ground_truth

[2.4832281428702565,
 0.0070204576626329285,
 -0.700242863196946,
 -0.9809447180873221,
 -1.103495567157565,
 -1.1554296976732905,
 -1.1726207581212915,
 -1.1719086561941543,
 -1.1618783799179462,
 -1.1472116880772685,
 -1.1305784577760987,
 -1.1135470606455158,
 -1.0970553900668067,
 -1.081663458927422,
 -1.0676938976979766]

In [17]:
experiment_truth = {bs:baseline(molecular_distances, basis_set=bs) for bs in basis_sets}
experiment_truth

{'sto-3g': [2.4832281428705083,
  0.007020457663510449,
  -0.7002428631962021,
  -0.9809447180873243,
  -1.103495567157565,
  -1.1554296976729574,
  -1.172620758121339,
  -1.1719086561941543,
  -1.1618783799179475,
  -1.1472116880772611,
  -1.130578457776084,
  -1.1135470606455198,
  -1.0970553900668043,
  -1.0816634589274265,
  -1.0676938976979757],
 '6-31g': [2.483228142870482,
  0.007020457663421631,
  -0.7002428631959507,
  -0.9809447180867119,
  -1.103495567157566,
  -1.1554296976734277,
  -1.172620758121339,
  -1.1719086561942027,
  -1.1618783799179462,
  -1.1472116880772611,
  -1.1305784577760876,
  -1.1135470606455187,
  -1.0970553900668023,
  -1.081663458927422,
  -1.0676938976979766]}

In [18]:
experiment_approximation = {bs:baseline(molecular_distances, basis_set=bs, numerical_method="hf") for bs in basis_sets}
experiment_approximation

{'sto-3g': [2.5214170199240216,
  0.045763856534167924,
  -0.661558784662049,
  -0.9424334409666507,
  -1.064869797930746,
  -1.1163487512431973,
  -1.1327453984944498,
  -1.130854719696118,
  -1.119249325983519,
  -1.1025764594670722,
  -1.0834608246367243,
  -1.0634232652500468,
  -1.0433556220180173,
  -1.0237798159149027,
  -1.004995582191915],
 '6-31g': [2.5214170199240344,
  0.04576385653418913,
  -0.6615587846620539,
  -0.9424334409666507,
  -1.064869797930739,
  -1.1163487512431929,
  -1.1327453984944629,
  -1.1308547196961227,
  -1.1192493259835237,
  -1.102576459467071,
  -1.0834608246367208,
  -1.0634232652500495,
  -1.043355622018018,
  -1.0237798159149032,
  -1.0049955821919168]}

# Running the VQE and uploading the results
For this demo, we will be using the `UCCSD` ansatz, with the `Nelder-mead` optimizer

In [ ]:
def run_uccsd_vqe(molecules, trotter_steps, silent=True):
    results = []
    optimizer = "Nelder-mead"
    qm_data = Qresult(optimizer)
    for i, molecule in enumerate(molecules):
        print(str(i+1)+"/"+str(len(molecules)), end="\t")
        print("Creating the Hamiltonian.", end="\t")
        H = molecule.make_hamiltonian()
        n_qubits = len(H.qubits)
        print("Creating ansatz.", end="\t")
        U = molecule.make_uccsd_ansatz(trotter_steps)
        #print("Creating objective function")
        E = tq.ExpectationValue(H=H, U=U)
        variables = {k:0.0 for k in U.extract_variables()}
        print("Optimizing", str(len(variables)), "vars.", end="\t")
        result = tq.minimize(objective=E, method=optimizer, initial_values=variables, silent=silent)
        print()
        results.append(result)
        qm_data.add_run(result, molecule, H, U)
    print("Done")
    #qm_data.push()
    return results

all_results = {" ".join([bs, tf]):[] for bs, tf in itertools.product(basis_sets, transformations)}
for bs, tf in itertools.product(basis_sets, transformations):
    all_results[" ".join([bs, tf])] = run_uccsd_vqe(molecules[" ".join([bs, tf])], 2)

1/15	Creating the Hamiltonian.	Creating ansatz.	Optimizing 1 vars.	
2/15	Creating the Hamiltonian.	Creating ansatz.	Optimizing 1 vars.	
3/15	Creating the Hamiltonian.	Creating ansatz.	Optimizing 1 vars.	
4/15	Creating the Hamiltonian.	Creating ansatz.	Optimizing 1 vars.	
5/15	Creating the Hamiltonian.	Creating ansatz.	Optimizing 1 vars.	
6/15	Creating the Hamiltonian.	Creating ansatz.	Optimizing 1 vars.	
7/15	Creating the Hamiltonian.	Creating ansatz.	Optimizing 1 vars.	
8/15	Creating the Hamiltonian.	Creating ansatz.	Optimizing 1 vars.	
9/15	Creating the Hamiltonian.	Creating ansatz.	Optimizing 1 vars.	
10/15	Creating the Hamiltonian.	Creating ansatz.	Optimizing 1 vars.	
11/15	Creating the Hamiltonian.	Creating ansatz.	Optimizing 1 vars.	
12/15	Creating the Hamiltonian.	Creating ansatz.	Optimizing 1 vars.	
13/15	Creating the Hamiltonian.	Creating ansatz.	Optimizing 1 vars.	
14/15	Creating the Hamiltonian.	Creating ansatz.	Optimizing 1 vars.	
15/15	Creating the Hamiltonian.	Creating an

# Plotting results to be shown on the Quantmark website

